In [1]:
import numpy as np
import cv2
import os
import sys

In [2]:
input_folder = "images"
output_folder = "output_images"

os.makedirs(output_folder, exist_ok=True)

In [8]:
img=cv2.imread("images/1.png")
img.shape


(512, 512, 3)

In [3]:
def detect_shape(cnt):
    peri = cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, 0.03 * peri, True)
    v = len(approx)

    if v == 3:
        return "Triangle"
    elif v == 4:
        return "Square"
    elif v >= 10:
        return "Star"
    elif v >= 6:
        return "Circle"
    else:
        return "Unknown"


In [4]:
def detect_color(cnt, hsv):
    mask = np.zeros(hsv.shape[:2], dtype="uint8")
    cv2.drawContours(mask, [cnt], -1, 255, -1)

    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.erode(mask, kernel, iterations=1)

    h, s, v, _ = cv2.mean(hsv, mask=mask)

    if s < 40 and v > 180:
        return "Grey"

    # People colors
    if h < 10 or h > 160:
        return "Red"
    elif 18 < h < 38:
        return "Yellow"
    elif 38 < h < 85:
        return "Green"

    # Camp colors
    elif 90 < h < 125:
        return "Blue"
    elif 125 < h < 155:
        return "Pink"

    return "Unknown"


In [17]:
for file_name in os.listdir(input_folder):

    input_path = os.path.join(input_folder, file_name)
    output_path = os.path.join(output_folder, file_name)

    image = cv2.imread(input_path)
    if image is None:
        continue

    image = cv2.resize(image, (800, 500))
    display = image.copy()


    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    land_lower = np.array([43, 110, 91])
    land_upper = np.array([77, 238, 161])
    land_mask = cv2.inRange(hsv, land_lower, land_upper)


    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (7, 7), 2)

    _, thresh = cv2.threshold(blur, 127, 255, cv2.THRESH_BINARY)

    thresh[land_mask == 255] = 0

    contours, _ = cv2.findContours(
        thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE
    )

    display[land_mask == 255] = (0, 200, 255)

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < 200:
            continue

        shape = detect_shape(cnt)
        color = detect_color(cnt, hsv)

        x, y, w, h = cv2.boundingRect(cnt)
        label = f"{color} {shape}"

        cv2.rectangle(display, (x, y), (x + w, y + h), (0, 255, 255), 1)
        cv2.putText(
            display, label, (x, y - 8),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 1)

    cv2.imwrite(output_path, display)
    print(f"Saved: {output_path}")

print("All images processed successfully.")


Saved: output_images\1.png
Saved: output_images\10.png
Saved: output_images\2.png
Saved: output_images\3.png
Saved: output_images\4.png
Saved: output_images\5.png
Saved: output_images\6.png
Saved: output_images\7.png
Saved: output_images\8.png
Saved: output_images\9.png
All images processed successfully.
